In [ ]:
# 🚀 ComfyUI Batch Image Generator for Real Estate Business

This notebook will:
1. Install ComfyUI on Google Colab
2. Download your models
3. Generate hundreds of product images
4. Package them for download

**⚡ Much faster than your Mac!**


In [ ]:
# 📦 Install ComfyUI and Dependencies
print("🚀 Installing ComfyUI...")

!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
!pip install -r requirements.txt
!pip install requests pillow

print("✅ ComfyUI installed!")


In [ ]:
# 📥 Download AI Models
print("📥 Downloading Stable Diffusion model...")

import os
os.makedirs('models/checkpoints', exist_ok=True)

# Download the model you were using
!wget -O models/checkpoints/v1-5-pruned-emaonly-fp16.safetensors \
  https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors

print("✅ Model downloaded!")
!ls -la models/checkpoints/


In [ ]:
# 🎨 Generate Real Estate Product Images

# Define your product prompts
product_prompts = [
    # Business Cards
    "Professional business cards, premium quality, clean white background, studio lighting, commercial photography",
    "Luxury business cards with metallic finish, high-end presentation, professional lighting",
    "Corporate business cards, standard quality, clean design, white background",
    "Real estate business cards, elegant design, professional presentation",
    "Premium business cards with embossed elements, luxury finish",
    
    # Flyers
    "Real estate flyers, professional design, property marketing materials, clean layout",
    "Property listing flyers, high-quality print materials, professional presentation",
    "Real estate marketing flyers, modern design, commercial photography",
    "Luxury property flyers, premium presentation, elegant design",
    "Open house flyers, professional real estate marketing materials",
    
    # Banners
    "Real estate banners, professional signage, outdoor advertising materials",
    "Property sale banners, high-quality vinyl printing, commercial grade",
    "Real estate yard signs, professional presentation, durable materials",
    "Open house banners, eye-catching design, professional quality",
    "Real estate advertising banners, premium materials, weather resistant",
    
    # Brochures
    "Real estate brochures, tri-fold design, professional presentation",
    "Property brochures, high-quality printing, elegant layout",
    "Luxury real estate brochures, premium paper stock, professional design",
    "Real estate marketing brochures, informational layout, clean design",
    "Property listing brochures, professional presentation, detailed information"
]

print(f"🎯 Ready to generate {len(product_prompts)} product images...")
print("📝 Run the next cell to start generation!")
